In [1]:
from finiteelementanalysis import visualize as viz
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis import solver_demo_helper_functions as solver_demo
from finiteelementanalysis.solver import hyperelastic_solver
import numpy as np
import matplotlib.pyplot as plt

This notebook will show that the FEA software (mostly) agrees with the analytical solutions to Euler beam approximations. Note that they will be slightly different since I could not find an analytical solution for a hyperelastic material.

We will be using D2_nn6_tri with 100 horizontal elements and 2 vertical elements. Our beam is represented by a 30 x 1 rectangle.

In [2]:
ele_type = "D2_nn6_tri" # change this!
nx = 100 # change this to refine the mesh
ny = 2 # change this to refine the mesh
L = 30
H = 1

coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)

In [3]:
# Identify boundaries
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0.0, L, 0.0, H)
# 1. Fix left boundary: both u_x and u_y = 0.
fixed_nodes_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)
fixed_nodes_right = pre.assign_fixed_nodes_rect(boundary_nodes, "right", 0.0, 0.0)
fixed_nodes = np.hstack((fixed_nodes_left, fixed_nodes_right))
# Assign distributed load on the right boundary
q = 2
dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, -q)
# Assign material properties
E = 1e6
nu = 0.3
# mu = E / (2.0 * (1.0 + nu))
# kappa = E / (3.0 * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))
#kappa = E / (2.0 * (1.0 - nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))
material_props = np.array([mu, kappa])
# Assign artificial displacement field
displacement = np.zeros((coords.shape))
for kk in range(0, coords.shape[0]):
    displacement[kk, 0] = coords[kk, 0] * 0.01

# Analytical solution
def solution(x, load_factor):
    w = q*load_factor
    I = H**3/12
    y = 0.5 - w * x**2 / (24*E*I) * (L-x)**2
    return y

# Animated plot
fname = "hw_solver_small_deformation.png"
pre_demo.plot_mesh_2D(fname, ele_type, coords, connect)

Our problem will have a distributed downward load on the top with both left and right ends fixed. Our load and material properties are defined below.

![alt text](small-def-analytic-solution.png)

In [7]:
# run the example to look at the results

nr_num_steps = 20
nr_print = True

displacements_all, nr_info_all = hyperelastic_solver(material_props, ele_type, coords.T, connect.T, fixed_nodes, dload_info, nr_print, nr_num_steps, nr_tol=1e-9, nr_maxit=30)

fname = "small-def.gif"
viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)

Step 0, load factor = 0.050
Iteration 1, Correction=1.000000e+00, Residual=1.111359e-04, tolerance=1.000000e-09
Iteration 2, Correction=2.379629e-05, Residual=6.163501e-05, tolerance=1.000000e-09
Iteration 3, Correction=2.711166e-10, Residual=8.363500e-13, tolerance=1.000000e-09
Step 1, load factor = 0.100
Iteration 1, Correction=5.000022e-01, Residual=1.111359e-04, tolerance=1.000000e-09
Iteration 2, Correction=9.792775e-06, Residual=6.163788e-05, tolerance=1.000000e-09
Iteration 3, Correction=3.870729e-12, Residual=7.778522e-13, tolerance=1.000000e-09
Step 2, load factor = 0.150
Iteration 1, Correction=3.333332e-01, Residual=1.111359e-04, tolerance=1.000000e-09
Iteration 2, Correction=8.080028e-06, Residual=6.163720e-05, tolerance=1.000000e-09
Iteration 3, Correction=9.528163e-11, Residual=8.459480e-13, tolerance=1.000000e-09
Step 3, load factor = 0.200
Iteration 1, Correction=2.499946e-01, Residual=1.111359e-04, tolerance=1.000000e-09
Iteration 2, Correction=8.580231e-06, Residual=6

![alt text](small-def.gif)

We now want to compare this to the analytical solution for Euler beam theory:

In [9]:
# Get the final displacements and new positions
final_disp = displacements_all[-1].reshape(-1, 2)
pos = coords + final_disp

# Plot the error
fig = plt.figure()
ax = fig.add_subplot()
ax.scatter(pos[:, 0], final_disp[:, 1] - (solution(pos[:, 0], 1) - 0.5))
ax.set_xlabel('x position')
ax.set_ylabel('Displacement error')
ax.set_title('Error Between Analytic & Numeric Solution')
plt.savefig('analytic-numeric-diff.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)

![alt text](analytic-numeric-diff.png)

It's pretty close overall, but there's a very clear pattern for the error. The error is ~5% which is still good, but even here we see the model starting to differentiate.